In [1]:
import json
import ast

from pyspark.sql import Row
from pyspark.sql import SparkSession 


In [2]:
spark = SparkSession \
       .builder \
       .enableHiveSupport() \
       .getOrCreate()
spark

## read from kafka

In [3]:
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.156.0.3:6667,10.156.0.4:6667,10.156.0.5:6667,") \
  .option("subscribe", "mles.announcements") \
  .load() 

In [4]:
df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



## Handling data

In [5]:
rdd = df.select("value").rdd#.show()

In [6]:
#rdd.take(10)

In [7]:
rdd_json = rdd.map(lambda x: ast.literal_eval(x.asDict()['value']))

In [8]:
def cur_dict(**x):
    if len(x) < 14:
        r_key = ['lng', 'description', 'floorsCount', 'price', 'dateInserted', 'roomsCount', 
                 'priceType', 'status', 'address', 'category', 'floorNumber', 'announcementid', 
                 'lat', 'totalArea']
        key = list(x.keys())
        for i in r_key:
            if not (i in key) :
                x[i] = 'Null'
    x['dadd'] = x['dateInserted'][:10]
    return Row(**x)

In [9]:
rdd_row = rdd_json.map(lambda x: cur_dict(**x))

In [10]:
df2 = spark.createDataFrame(rdd_row)
df2.collect()

[Row(address='Краснодарский край, городской округ Анапа, Анапа, ул Ленина, 183', announcementid='212349699', category='flatSale', dadd='2019-07-31', dateInserted='2019-07-31T16:49:28.018956+03:00', description='Объект 29881. Продаю квартиру с видом на море, общей площадью 78 кв. м. Две спальни (изолированные) и просторная гостиная, совмещенная с кухней, с выходом на застекленную лоджию (все окна на море). Санузел раздельный, центральное отопление, газ, сплит системы во всех комнатах. В квартире новый ремонт под ключ, учли все мелочи т. к. делали для себя! Мебелеруйте на свой вкус и квартира в престижном доме готова! Отличный огороженный двор, парковка, детская площадка. Пять лет в собственности, один собственник, свидетельство на руках!', floorNumber='8', floorsCount='10', lat=44.871883, lng=37.330696, price=4600000.0, priceType='all', roomsCount='2', status=1, totalArea=78.0),
 Row(address='Краснодарский край, городской округ Анапа, Анапа, ул Стахановская, 16', announcementid='2123496

## Write to hive

In [11]:
df2.write.format('orc').mode('overwrite').partitionBy('dadd').saveAsTable('sschokorov.test_pow')

In [12]:
#df2.printSchema()